In [1]:
import re, json, pathlib, logging, time, argparse, pprint, warnings
import datetime as dt
import importlib.resources as rsrc
from itertools import chain
from operator import itemgetter 
#
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk as nltk
from nltk.corpus import stopwords
from whoosh.lang.porter import stem
#from googletrans import Translator
from pygoogletranslation import Translator
import circlify as crcf
import matplotlib.pyplot as plt
import random

import polmap.polmap as plmp
import polmap.clean_docs as cldc
import postprocess.postprocess as pspr
from docx2python import docx2python
import docx2python.iterators as d2piter


##MM imports
#from polmap.polmap import make_directories, preprocess_text, doc2text, SDGrefs_mapper # replaced the keyword processing block

#import fromRtoPython

from docx2python import docx2python
 
#from polmap.polmap import make_directories, preprocess_text, doc2text # replaced the keyword processing block

In [2]:
term_df=pd.read_excel('keywords/term_matrix.xlsx', index_col=0)

In [3]:
term_df.head()

,Goal,Target,0,1,2,3,4,5,6,7,...,48,49,50,51,52,53,54,55,56,57
0,SDG 1,1.0,Sustainable Development Goal 1,SDG 1,end poverty,reduce poverty,fight against poverty,end global poverty,poverty reduction,poverty and deprivation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SDG 1,1.1,international poverty line,extreme poverty,abject poverty,absolute poverty,destitution,penury,severe deprivation,help people suffering from food and material d...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SDG 1,1.2,severely materially deprived people,low work intensity,national poverty line,social exclusion,income poverty,combatting child poverty,children out of poverty,addressing child poverty,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SDG 1,1.3,social risk,social protection,rights of persons with disabilities,rights of unemployed persons,rights of older persons,rights of pregnant women,rights of work-injury victims,rights of the poor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SDG 1,1.4,tenure rights,equal rights to economic resources,control over land,access to inheritance,rights to land,\naccess to property,access to natural resources,access to new technology,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
keywords_df = {}
for row in term_df.itertuples():
    sheet=pd.DataFrame(data={'Terms':row})
    sheet['Goal'] = row.Goal
    sheet['Target'] = row.Target
    sheet = sheet.reindex(columns=['Goal','Target','Terms'])
    keywords_df[row.Goal] = sheet   


In [12]:
pd.DataFrame().to_excel('keywords_for_translation.xlsx')

for key, value in keywords_df.items():
    with pd.ExcelWriter('keywords_for_translation.xlsx', mode='a', engine='openpyxl') as destfile:
        value.to_excel(destfile, sheet_name=key)
